# SV database

In [6]:
libs <- c("data.table","dplyr","igraph","parallel")
for (i in libs){
  if( !is.element(i, .packages(all.available = TRUE)) ) 
  {
    if(is.element(i,available.packages()[,1])){install.packages(i)}
    else{
    source("https://bioconductor.org/biocLite.R")
    biocLite(i);}
  }
  library(i,character.only = TRUE)
}





## 读取文件

In [7]:
DGV_Supporting<-fread(input = "/media/_EXTend2018/Taoyiran/File_149/SVdatabase/GRCh37_hg19_supportingvariants_2016-05-15.txt")
DGV_Supporting<-DGV_Supporting[-grep("9_gl000200_random",DGV_Supporting$chr),]
DGV_Supporting<-DGV_Supporting[DGV_Supporting$mergedorsample=="S",]
DGV_Information<-fread("/media/_EXTend2018/Taoyiran/File_149/jupyter/DGV_infomation.txt")
DGV_Supporting$Ethnicity<-DGV_Information[match(DGV_Supporting$samples,DGV_Information$`external sample id`),"ethnicity_New"]

Read 6668715 rows and 20 (of 20) columns from 0.890 GB file in 00:00:13


补全gain/loss为NA的数据

In [8]:
temp<-grep(TRUE,is.na(DGV_Supporting[,"observedgains"]))
#DGV_Supporting[temp,"variantsubtype"]%>%table
DGV_Supporting[intersect(temp,grep(TRUE,DGV_Supporting[,"variantsubtype"]=="deletion")),"observedgains"]<-c(0)
DGV_Supporting[intersect(temp,grep(TRUE,DGV_Supporting[,"variantsubtype"]=="deletion")),"observedlosses"]<-c(1)

DGV_Supporting[intersect(temp,grep(TRUE,DGV_Supporting[,"variantsubtype"]=="insertion")),"observedgains"]<-c(1)
DGV_Supporting[intersect(temp,grep(TRUE,DGV_Supporting[,"variantsubtype"]=="insertion")),"observedlosses"]<-c(0)

DGV_Supporting[intersect(temp,grep(TRUE,DGV_Supporting[,"variantsubtype"]=="loss")),"observedgains"]<-c(0)
DGV_Supporting[intersect(temp,grep(TRUE,DGV_Supporting[,"variantsubtype"]=="loss")),"observedlosses"]<-c(1)


### 一号染色体

In [9]:
DGV_Supporting_chr1<-DGV_Supporting[DGV_Supporting$chr=="22",]
DGV_Supporting_START.END<-DGV_Supporting_chr1[,c("variantaccession","chr","start","end","observedgains","observedlosses","supportingvariants","variantsubtype","genes","samples","Ethnicity")]
DGV_Supporting_START.END<-dplyr::arrange(DGV_Supporting_START.END,by_group=start)
DGV_Supporting_START.END[,3]<-DGV_Supporting_START.END[,3]%>%as.numeric
DGV_Supporting_START.END[,4]<-DGV_Supporting_START.END[,4]%>%as.numeric

### Split into Blocks

In [10]:
ClusterByPos=function(vec,min.dis,multi.samples = FALSE)
{
names(vec) = 1:length(vec)
sort.vec = sort(vec)
diff.sort.vec = diff(sort.vec)
split.index = which(diff.sort.vec > min.dis)
start.index = c(1,split.index + 1)
end.index = c(split.index,length(sort.vec))
cls.list = lapply(apply(cbind(start.index,end.index),1,function(x){y = list(as.numeric(names(sort.vec)[x[1]:x[2]])) }),unlist)
cls.list
}

start.index<-ClusterByPos(DGV_Supporting_START.END$start,100)
end.index<-ClusterByPos(DGV_Supporting_START.END$end,100)

ClusterBoth<-function(start.index,end.index)
{
names(start.index)<-c(1:length(start.index))
names(end.index)<-c(1:length(end.index))

start.unlist<-unlist(start.index)
names(start.unlist)<-unlist(lapply(1:length(start.index),function(x){rep(x,length(start.index[[x]]))}))

end.unlist<-unlist(end.index)
names(end.unlist)<-unlist(lapply(1:length(end.index),function(x){rep(x,length(end.index[[x]]))}))
end.unlist<-sort(end.unlist)


Index<-paste(names(start.unlist),names(end.unlist),sep = "-")
names(Index)<-c(1:length(Index))
Index_2<-data.frame(cbind(Index,names(Index)))
index_3<-group_indices(Index_2, Index)
return(final.list<-ClusterByPos(index_3,0))

}
Block.list<-ClusterBoth(start.index,end.index)


In [11]:
#判断每个block是否需要做Igraph 下回使用tapply
DistOfBlock<-function(block.list)
  { 
  if(length(block.list)==1){return(c("No"))}
  else{
      x<-DGV_Supporting_START.END[block.list,c(1,3,4)]
      a<-abs(max(x[,2])-min(x[,2]))
      b<-abs(max(x[,3])-min(x[,3]))
      c<-max(a,b)
     # print(block.list[1])
      if(c>100){return(c("Need"))}
      else{return(c("No"))}
     # print(block.list[1])
      }
}
Block.needIgraph<-lapply(Block.list,DistOfBlock)
NeedOrNot<-unlist(Block.needIgraph)
Block.needIgraph<-Block.list[NeedOrNot=="Need"]
Block.NO.needIgraph<-Block.list[NeedOrNot=="No"]

### 多进程生成距离矩阵

In [12]:
#多进程生成距离矩阵
library(parallel)
cl <- makeCluster(5)

Block2Dist<-function(block.list)
	{
    x<-DGV_Supporting_START.END[block.list,c(3:4)]
  	e<-c(rep(0,nrow(x)))
  	for(i in 1:nrow(x))
		{
		d<-c(rep(0,nrow(x)))
		for(j in 1:nrow(x))
			{
			if(i==j)
				{
				d[j]<-c(0)
				}
			else
				{
				a<-abs(x[i,1]-x[j,1])
				b<-abs(x[i,2]-x[j,2])
				d[j]<-max(a,b)
				}
			}
		e<-rbind(e,d)	
		}
	e<-e[-1,]
	return(e)
	}

clusterExport(cl, "DGV_Supporting_START.END")  
system.time({BlockToDist<- parLapply(cl,Block.needIgraph,Block2Dist)})


#DistTest<- do.call('rbind',res)

for(i in 1:length(BlockToDist))
{
  rownames(BlockToDist[[i]])<-Block.needIgraph[[i]]
  colnames(BlockToDist[[i]])<-Block.needIgraph[[i]]
 # print(i)
}
#save(BlockToDist,file="BlockToDist_chr1.Rdata")

    user   system  elapsed 
   0.561    0.406 1072.718 

&nbsp;
### 把距离矩阵转为01矩阵（多进程）

In [13]:
library(parallel)
cl <- makeCluster(5)
clusterExport(cl, "DGV_Supporting_START.END")  
DistTo0or1<-function(x)
  {
DistTest<-x
DistTest[DistTest<=100]<-c("1")
DistTest[DistTest>100]<-c("0")
DistTest[row(DistTest)==col(DistTest)]<-c("0")
return(DistTest)
}

#BlockToDist<-lapply(BlockToDist,DistTo0or1)
system.time({BlockToDist_0or1<-parLapply(cl,BlockToDist,DistTo0or1)})


   user  system elapsed 
  7.847   0.928  22.388 

&nbsp;
### 通过01矩阵生成Block

In [14]:
Block2graph<-function(BlockToDist_0or1){
temp<-list()
for(i in 1:length(BlockToDist_0or1))
{
a<-graph.adjacency(BlockToDist_0or1[[i]],mode="undirected")
a<-max_cliques(a,min = 1)
temp<-c(temp,list(a))
#print(i)
}
return(temp)
}

system.time({Block.cliques<-Block2graph(BlockToDist_0or1)})

#unlist(Block.cliques[[2]])[duplicated(unlist(Block.cliques[[2]]))]

#do.call(sum,lapply(lapply(Block.cliques,function(x){length(x)}),sum)) #看一共有多少clique 5744


   user  system elapsed 
577.391   5.351 584.133 

&nbsp;
### 删除重复的SV

In [15]:
CliquesRMduplicated<-function(BlockCliques)
{
temp<-lapply(BlockCliques,length)%>%unlist
names(temp)<-c(1:length(temp))
temp2<-BlockCliques[temp%>%sort(decreasing=TRUE)%>%names%>%as.numeric]

temp3<-list()
for(i in 1:length(temp2))
	{
	temp3<-c(temp3,list(names(unlist(temp2[[i]]))))
	names(temp3[[i]])<-rep(i,temp2[[i]]%>%length)
	}
temp4<-unlist(temp3)
temp4<-temp4[duplicated(temp4)!=TRUE]
temp5<-list()
for(j in unique(temp4%>%names))
	{
	temp6<-temp4[temp4%>%names==j]
	temp5<-c(temp5,list(temp6))
	}

return(temp5)	
}
BlocK.RMdupli.clique<-lapply(Block.cliques,CliquesRMduplicated)

#do.call(sum,lapply(lapply(BlocK.RMdupli.clique,function(x){length(x)}),sum))
#4887个cliques

<font color=red>**把BlocK.RMdupli.clique转为单层list**</font>

In [16]:
t<-NULL
for(i in 1:length(BlocK.RMdupli.clique))
{
t<- c(t,BlocK.RMdupli.clique[[i]])
}
BlocK.RMdupli.clique<-t
names(BlocK.RMdupli.clique)<-1:length(BlocK.RMdupli.clique)
rm(t)

&nbsp;
&nbsp;
&nbsp;
## 对应Information


In [17]:
Merge_cliques<-function(cliques)
{
    info<-NULL
	temp<-DGV_Supporting_START.END[unlist(cliques)%>%as.numeric,]
	temp<-temp[grep(FALSE,temp[,"samples"]%>%unlist%>%duplicated),]
    
    
    
    
    
    info2<-c(temp[,"chr"]%>%unique,temp[,"start"]%>%min,temp[,"end"]%>%max)
	supportingvariants<-paste(temp[,"variantaccession"],collapse=",")
	variantsubtype<-temp[,"variantsubtype"]%>%unique%>%paste(collapse = ",")
	
     
    observedgains<-temp[,"observedgains"]%>%sum
	observedlosses<-temp[,"observedlosses"]%>%sum
	genes<-temp[,"genes"]%>%strsplit(",")%>%unlist%>%unique%>%paste(collapse = ",")
	samples<-temp[,"samples"]%>%unlist%>%paste(collapse = ",")
    
    ethnicity<-temp[,"Ethnicity"]%>%table
    
    European_Non_Finnish<-ethnicity["European (Non-Finnish)"]
		European_Finnish<-ethnicity["European Finnish"]
		East_Asian<-ethnicity["East Asian"]
		Latino<-ethnicity["Latino"]
		South_Asian<-ethnicity["South Asian"]
		African<-ethnicity["African"]
		No_report<-ethnicity["No-report"]
		Other<-ethnicity["Other"]
		Ashkenazi_Jewish<-ethnicity["Ashkenazi Jewish"]
		ethnicity_new<-c(European_Non_Finnish,European_Finnish,East_Asian,Latino,South_Asian,African,No_report,Other,Ashkenazi_Jewish)
		ethnicity_new[ethnicity_new%>%is.na]=0
    
    #samples_duplicated<-temp[grep(TRUE,temp[,"samples"]%>%unlist%>%duplicated),]
  #  observedgains<-observedgains-samples_duplicated[,"observedgains"]%>%as.numeric
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
        info2<-c(info2,supportingvariants,variantsubtype,observedgains,observedlosses,genes,samples,ethnicity_new)
        info2<-rbind(info,info2)
    return(info2)
}
system.time({Variants_Merge<-lapply(BlocK.RMdupli.clique,Merge_cliques)})


   user  system elapsed 
  7.360   0.053   7.431 

In [18]:
system.time({Variants_Nograph_Merge<-lapply(Block.NO.needIgraph,Merge_cliques)})

   user  system elapsed 
 44.054   0.303  44.463 

In [19]:
temp<-do.call(rbind,Variants_Merge)
temp2<-do.call(rbind,Variants_Nograph_Merge)
temp<-rbind(temp,temp2)
rownames(temp)<-1:nrow(temp)
colnames(temp)<-c("chr","start","end","supportingvariants","variantsubtype","observedgains","observedlosses","genes","samples","European_Non_Finnish","European_Finnish","East_Asian","Latino","South_Asian","African","No_report","Other","Ashkenazi_Jewish")

In [20]:
write.table(temp,file = "chr22.txt",quote = FALSE,sep = "\t",row.names = FALSE,col.names = TRUE)

In [21]:
temp%>%dim

[1] 10662    18

In [22]:
ls()

[1] "Block.cliques"            "Block.list"              
 [3] "Block.needIgraph"         "Block.NO.needIgraph"     
 [5] "BlocK.RMdupli.clique"     "Block2Dist"              
 [7] "Block2graph"              "BlockToDist"             
 [9] "BlockToDist_0or1"         "cl"                      
[11] "CliquesRMduplicated"      "ClusterBoth"             
[13] "ClusterByPos"             "DGV_Information"         
[15] "DGV_Supporting"           "DGV_Supporting_chr1"     
[17] "DGV_Supporting_START.END" "DistOfBlock"             
[19] "DistTo0or1"               "end.index"               
[21] "i"                        "libs"                    
[23] "Merge_cliques"            "NeedOrNot"               
[25] "start.index"              "temp"                    
[27] "temp2"                    "Variants_Merge"          
[29] "Variants_Nograph_Merge"